<a href="https://colab.research.google.com/github/talhaanwarch/OffenseEval2020/blob/master/Danish/FastText_biGRU_crossval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Danish Language


In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install focal-loss
!pip install keras-radam

  Created wheel for keras-radam: filename=keras_radam-0.15.0-cp36-none-any.whl size=14685 sha256=3decc46c559dc4b58470c6119799d078bce606820e413d467001f925681cdc69
  Stored in directory: /root/.cache/pip/wheels/79/a0/c0/670b0a118e8f078539fafec7bd02eba0af921f745660c7f83f
Successfully built keras-radam


In [3]:
cd /content/drive/My Drive/dataset/OffenseEval2020/data/Danish/

/content/drive/My Drive/dataset/OffenseEval2020/data/Danish


In [4]:
ls

cc.da.300.vec  cc.da.300.vec.gz  offenseval-da-training-v1.tsv


In [0]:
import pandas as pd
train=pd.read_csv( 'offenseval-da-training-v1.tsv',sep="\t")
train.drop(train.tail(n=1).index,inplace=True)
#train['tweet']=train['tweet'].str.lower()
train=train.sample(frac=1)

In [6]:
train.head()

,id,tweet,subtask_a
612,584,Igen fylder godt i statistikkerne.,NOT
224,878,Synes ikke velgørenhed firmaer i dk er værd at...,NOT
202,2798,Beriget!,NOT
277,365,@USER sikke en lille ting at blive sur over? D...,NOT
1493,1753,Det är för att vi alla redan kan engelska.,NOT


In [7]:
train.tail()

,id,tweet,subtask_a
218,3419,[He he he...](URL,NOT
2844,1436,Mon det bliver oplyst til skat ?????,NOT
829,981,Ja hvorfor ikk tjene på det man er blevet give...,NOT
1399,2598,Lel hvor har du fundet den? Det er vidst det j...,OFF
1906,136,Bliver bange,NOT


In [0]:
labels=train['subtask_a']
train=train['tweet']
labels=pd.factorize(labels)[0]

In [9]:
import collections
collections.Counter(labels)

Counter({0: 2576, 1: 384})

In [10]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
max_words = 10000 #frequency of words to be kept
max_len = 200

tokenize = Tokenizer(num_words=max_words)
tokenize.fit_on_texts(train)
sequences = tokenize.texts_to_sequences(train)
word_index = tokenize.word_index
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

Using TensorFlow backend.


In [11]:
len(sequences_matrix),len(labels)

(2960, 2960)

In [0]:
from keras.models import Sequential
from keras.layers import Embedding,CuDNNGRU,GRU,Dense,Dropout,Bidirectional,GlobalMaxPool1D,GlobalAveragePooling1D, SpatialDropout1D
from keras.optimizers import RMSprop
from keras.initializers import Constant
from sklearn.utils import class_weight
from sklearn.metrics import classification_report,f1_score
import keras
import numpy as np
from sklearn.model_selection import StratifiedKFold
from focal_loss import BinaryFocalLoss
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

In [13]:
num_words = min(max_words, len(word_index)) + 1
print(num_words)

9970


In [0]:
# !wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.el.300.vec.gz

In [0]:
# import gzip
# import shutil
# with gzip.open('cc.el.300.vec.gz', 'rb') as f_in:
#     with open('cc.el.300.vec', 'wb') as f_out:
#         shutil.copyfileobj(f_in, f_out)

In [0]:
embed_size=300
def get_coefs(word,*arr):
    return word, np.asarray(arr, dtype='float32')

def build_matrix(embedding_path, word_index):
    embedding_index = dict(get_coefs(*o.strip().split(" ")) for o in open(embedding_path))

    nb_words = min(max_words, len(word_index))
    embedding_matrix = np.zeros((nb_words + 1, embed_size))
    for word, i in word_index.items():
        if i >= max_words:
            continue
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [0]:
embeddings=build_matrix('cc.da.300.vec', word_index)


In [18]:
embeddings.shape

(9970, 300)

In [0]:
def model_func():
  model = Sequential()
  model.add(Embedding(num_words,embed_size,embeddings_initializer=Constant(embeddings),input_length=max_len,trainable=False))
  model.add(Bidirectional(CuDNNGRU(100, return_sequences = True)))
  #model.add(Bidirectional(CuDNNGRU(32, return_sequences = True)))
  model.add(GlobalMaxPool1D())
  model.add(Dense(100, activation="relu"))
  model.add(Dense(50, activation="relu"))
  #model.add(Dense(10, activation="relu"))

  model.add(Dropout(0.05))
  model.add(Dense(1, activation="sigmoid"))
  model.compile(loss=BinaryFocalLoss(gamma=2), optimizer='adam', metrics=['accuracy'])
  return model

In [0]:
import numpy as np
class_weights = class_weight.compute_class_weight('balanced',np.unique(labels),labels)
class_weights=dict(enumerate(class_weights))

In [0]:
from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2, min_lr=0.000001, verbose=1)

In [22]:
kfold = StratifiedKFold(n_splits=10, shuffle=False, random_state=2020)
cvscores = []

for train, test in kfold.split(sequences_matrix, labels):
  model=model_func()
  
  model.fit(sequences_matrix[train], labels[train],batch_size=256,epochs=10,verbose=2,
            validation_data=(sequences_matrix[test],labels[test]),callbacks=[reduce_lr])
  y_pred = model.predict(sequences_matrix[test], batch_size=128, verbose=1)
  y_pred_bool = np.argmax(y_pred, axis=1)
  y_pred = (y_pred > 0.5)
  f1=f1_score(labels[test], y_pred, average='macro')
  cvscores.append(f1)
  keras.backend.clear_session()

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Train on 2664 samples, validate on 296 samples
Epoch 1/10
 - 15s - loss: 0.1241 - acc: 0.8315 - val_loss: 0.1140 - val_acc: 0.8716
Epoch 2/10
 - 1s - loss: 0.1082 - acc: 0.8701 - val_loss: 0.1037 - val_acc: 0.8716
Epoch 3/10
 - 1s - loss: 0.1024 - acc: 0.8701 - val_loss: 0.1011 - val_acc: 0.8716
Epoch 4/10
 - 1s - loss: 0.0951 - acc: 0.8701 - val_loss: 0.0992 - val_acc: 0.8716
Epoch 5/10
 - 1s - loss: 0.0874 - acc: 0.8701 - val_loss: 0.0948 - val_acc: 0.8716
Epoch 6/10
 - 1s - loss: 0.0772 - acc: 0.8701 - val_loss: 0.0914 - val_acc: 0.8716
Epoch 7/10
 - 1s - loss: 0.0694 - acc: 0.8825 - val_loss: 0.0910 - val_acc: 0.8851
Epoch 8/10
 - 1s - loss: 0.0618 - acc: 0.9084 - val_loss: 0.0962 - val_acc: 0.8885
Epoch 9/10
 - 1s - loss: 0.0554 - acc: 0.9197 - val_loss: 0.0983 - val_acc: 0.8885

Epoch 00009: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
Epoch 10/10
 - 1s - loss: 0.0480 - acc: 0.9328 - val_loss: 0.0976 - val_acc: 0.8818
296/296 [==============================

In [23]:
np.array(cvscores).mean()

0.7032963451515238